<a href="https://colab.research.google.com/github/Lausti98/bsc-recsys/blob/main/bsc_book_crossing_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install recpack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import required module
from pathlib import Path
import pandas as pd
import re
from sklearn import preprocessing
fpath = 'BX-Book-Ratings.csv'

df = pd.read_csv(fpath, sep=';', on_bad_lines='skip', encoding_errors='replace')#, error_bad_lines=False, skiprows=190000)
le = preprocessing.LabelEncoder()
df['ISBN'] = le.fit_transform(df['ISBN'])
df['User-ID'] = le.fit_transform(df['User-ID'])
print(df.dtypes)
df


User-ID        int64
ISBN           int64
Book-Rating    int64
dtype: object


,User-ID,ISBN,Book-Rating
0,104433,57188,0
1,104434,29750,5
2,104435,107392,0
3,104436,127253,3
4,104436,127287,6
...,...,...,...
1149775,104428,256876,9
1149776,104429,164861,0
1149777,104430,123711,10
1149778,104431,145625,10


In [ ]:
from recpack.preprocessing.preprocessors import DataFramePreprocessor
from recpack.preprocessing.filters import MinItemsPerUser, MinUsersPerItem

proc = DataFramePreprocessor(item_ix='ISBN', user_ix='User-ID')#, timestamp_ix='ts')
proc.add_filter(MinUsersPerItem(5, item_ix='ISBN', user_ix='User-ID'))
proc.add_filter(MinItemsPerUser(5, item_ix='ISBN', user_ix='User-ID'))

# Assuming you have loaded a dataframe called df
interaction_matrix = proc.process(df)

  0%|          | 0/607651 [00:00<?, ?it/s]

  0%|          | 0/607651 [00:00<?, ?it/s]

In [ ]:
from recpack.scenarios import WeakGeneralization

# Split data into train, test and validation set. Random split not accounting for time data.
scenario = WeakGeneralization(0.75, validation=True)
scenario.split(interaction_matrix)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
from recpack.pipelines import PipelineBuilder

builder = PipelineBuilder()

builder.set_data_from_scenario(scenario)

In [ ]:
builder.add_algorithm('Popularity') # No real parameters to optimise

In [ ]:
# ITEM BASED Nearest Neighbors Recommendation Recommender Systems: An Introduction 2.2
builder.add_algorithm('ItemKNN', grid={
    'K': [100, 200, 500],
    'similarity': ['cosine'],# 'conditional_probability'],
})

#builder.add_algorithm('SLIM')

#builder.add_algorithm('KUNN', grid={
#    'Ku': [100, 200, 500],
#    'Ki': [100, 200, 500],
#})

/usr/local/lib/python3.8/dist-packages/recpack/pipelines/pipeline_builder.py:145: UserWarning: Grid parameter for add_algorithm function will be deprecated in favour of optimisation_info.
  warnings.warn(


In [ ]:
builder.set_optimisation_metric('NDCGK', K=10)
builder.add_metric('NDCGK', K=[10, 20, 50])
builder.add_metric('CoverageK', K=[10, 20])
builder.add_metric('RecallK', K=[10, 20, 50])
builder.add_metric('PrecisionK', K=[10, 20, 50])

In [ ]:
pipeline = builder.build()
pipeline.run()
pipeline.get_metrics()

  0%|          | 0/4 [00:00<?, ?it/s]

2023-03-02 20:23:28,220 - base - recpack - INFO - Fitting Popularity complete - Took 0.343s


INFO:recpack:Fitting Popularity complete - Took 0.343s
/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


2023-03-02 20:23:48,803 - base - recpack - INFO - Fitting ItemKNN complete - Took 10.1s


/usr/local/lib/python3.8/dist-packages/recpack/algorithms/base.py:279: UserWarning: ItemKNN missing similar items for 479 items.
  warnings.warn(f"{self.name} missing similar items for {missing} items.")
INFO:recpack:Fitting ItemKNN complete - Took 10.1s


...

INFO:recpack:Fitting ItemKNN complete - Took 14.3s


2023-03-02 20:24:49,152 - base - recpack - INFO - Fitting ItemKNN complete - Took 33.8s


INFO:recpack:Fitting ItemKNN complete - Took 33.8s


2023-03-02 20:25:17,620 - base - recpack - INFO - Fitting ItemKNN complete - Took 12.5s


/usr/local/lib/python3.8/dist-packages/recpack/algorithms/base.py:279: UserWarning: ItemKNN missing similar items for 101 items.
  warnings.warn(f"{self.name} missing similar items for {missing} items.")
INFO:recpack:Fitting ItemKNN complete - Took 12.5s


In [ ]:
print(df['User-ID'].nunique())
print(df['ISBN'].nunique())